#### Payroll Attendacne - Amit (Original)

In [137]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [138]:
import warnings
warnings.filterwarnings("ignore", message=".*SQLAlchemy connectable.*")

In [139]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:\Users\Administrator\LIS\Kenvue\python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [140]:
# Python Master for ISPStore
sheet = client.open_by_key("1T_fuAUdJw-DB9dn9v3ZRg5YZ3mruCtg3Ghkd1mhVBRM") # Master Sheets ( Dimensions Sheet)

In [141]:
Schema_Master = sheet.worksheet("Schema_Master")
Schema_Master = get_as_dataframe(Schema_Master)
Data = sheet.worksheet("Data")

In [142]:
Schema_Master= Schema_Master[Schema_Master['flag']==1]
Schema_Master.head()

Project            SQL Schema Name  From Date    To Date  flag  \
0    Amul GT & MT           [AmulManagement]  21-Aug-25  20-Sep-25   1.0   
1     AWL Agri-GT    [AdaniWilmerManagement]  16-Aug-25  15-Sep-25   1.0   
2  Bacardi Legacy  [BacardilegacyManagement]  01-Sep-25  30-Sep-25   1.0   
3        Bajaj MT        [BajajMTManagement]  26-Aug-25  25-Sep-25   1.0   
4  Bajaj Consumer  [BajajConsumerManagement]  26-Aug-25  25-Sep-25   1.0   

   flag2 Unnamed: 7 Unnamed: 8 Unnamed: 9  
0    1.0     =C2+31     =D2+31          d  
1    1.0     =C3+31     =D3+31          d  
2    1.0     =C4+31     =D4+30          d  
3    1.0     =C5+31     =D5+31          d  
4    1.0     =C6+31     =D6+31          d

In [143]:
x= Schema_Master['SQL Schema Name'].to_list()
projects = [str(p).strip("[]") for p in Schema_Master['Project'].tolist()]


In [144]:
date_ranges = [
    (
        pd.to_datetime(row["From Date"]).strftime("%Y-%m-%d"),
        pd.to_datetime(row["To Date"]).strftime("%Y-%m-%d")
    )
    for _, row in Schema_Master.iterrows()
]


In [145]:
server = "157.119.230.120,4071"
database = "PowerBI_LISV2"
username = "sa"
password = "Y0m@Sql.!.123"

conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
TrustServerCertificate=yes;
"""

In [146]:
conn = pyodbc.connect(conn_str)

In [147]:
# x = ['[CholayilManagement]','[ParleManagement]']

In [148]:
final_df = pd.DataFrame()

In [149]:
# date_ranges = [
#     ("2025-07-21", "2025-07-22"),
#      ("2025-08-22","2025-08-24")
# ]

In [150]:
pivot_frames = []

In [151]:
for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
        query = f""" 
WITH CTE AS (
    SELECT
        '{project}' AS SchemaName,
        A.UserName,
        M.EmployeeID,
        M.FullName,
        M.Gender,
        M.Region,
        M.State,
        M.City,
        M.Designation,
        TRY_CONVERT(date, M.LastWorkingDate, 105) AS LastWorkingDate,
        M.Status AS EmpStatus,
        -- A.Status AS AttRawStatus,
        -- A.ApprovalStatus,
        CASE
            WHEN A.Status = 'Present' THEN 'P'
            WHEN A.Status = 'Absent' OR A.Status IS NULL THEN 'A'
            WHEN A.Status = 'Leave' THEN 'L'
            WHEN A.Status IN ('Compensatory Off','Comp Off') THEN 'CO'
            WHEN A.Status IN ('Week Off','WeekOff') THEN 'WO'
            WHEN A.Status IN ('Meeting','Meeting/Training') THEN 'M'
            WHEN A.Status = 'Training' THEN 'T'
            WHEN A.Status IN ('National Holiday','National Hoilday','Holiday','Regional Holiday') THEN 'H'
            WHEN A.Status IN ('Bandh/Strike','Bandh Strike','Bandh / Strike','Market closed','Strike') THEN 'B'
            WHEN A.Status = 'Vacant' THEN 'V'
            WHEN A.Status IN ('Personal','Personal Leave') THEN 'PL'
            WHEN A.Status = 'Casual Leave' THEN 'CL'
            WHEN A.Status = 'Half Day' THEN 'HD'
            ELSE 'Condition not found'
        END AS AttStatus,
        DATEDIFF(
            DAY, 
            CAST('{date_from}' AS DATE), 
            TRY_CONVERT(date, A.[Date], 105)
        ) + 1 AS DayNum, 
        -- TRY_CONVERT(date, A.[Date], 105) AS Date2,
        RANK() OVER (PARTITION BY A.UserName,A.[Date] ORDER BY A.ISPAttendanceMasterExportId DESC) AS rk
    FROM PowerBI_LISV2.{value}.ISPAttendanceMasterExport AS A
    LEFT JOIN (SELECT K.* FROM
        (SELECT  *, RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK FROM 
        PowerBI_LISV2.{value}.UserMaster) AS K
        WHERE K.RNK =1) AS M
        ON A.UserName = M.UserName
    WHERE 
        TRY_CONVERT(date, A.[Date], 105) BETWEEN '{date_from}' AND '{date_to}'
        AND M.UserRole NOT IN ('MIS','RSM','SO','Program Manager','Assistant Manager','FO','Office Staff','ASM',
        'Senior Manager','Supervisor','National')
        AND LOWER(A.UserName) NOT LIKE '%test%'
        AND LOWER(A.UserName) NOT LIKE '%sup%'
        AND LOWER(A.FullName) NOT LIKE '%test%'
),
Filtered AS (
    SELECT *
    FROM CTE
    WHERE rk = 1
),
Pivoted AS (
    SELECT
        SchemaName,
        UserName,
        EmployeeID,
        FullName,
        Gender,
        Region,
        State,
        City,
        Designation,
        LastWorkingDate,
        EmpStatus,
        -- day columns produced by the pivot
        [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
        [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
        [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
    FROM Filtered
    PIVOT (
        MAX(AttStatus) FOR DayNum IN (
            [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
            [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
            [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
        )
    ) AS P
)
SELECT *
FROM Pivoted;"""
        df = pd.read_sql(query, conn)
        final_df = pd.concat([final_df, df], ignore_index=True)
        print(f"Running for: {value} {date_from} {date_to}")
           

Running for: [AmulManagement] 2025-08-21 2025-09-20
Running for: [AdaniWilmerManagement] 2025-08-16 2025-09-15
Running for: [BacardilegacyManagement] 2025-09-01 2025-09-30
Running for: [BajajMTManagement] 2025-08-26 2025-09-25
Running for: [BajajConsumerManagement] 2025-08-26 2025-09-25
Running for: [IDAMManagement] 2025-08-26 2025-09-25
Running for: [BrillonMTManagement] 2025-08-16 2025-09-15
Running for: [CamlinManagement] 2025-08-26 2025-09-25
Running for: [CholayilManagement] 2025-08-21 2025-09-20
Running for: [DaburPharmaManagement] 2025-09-01 2025-09-30
Running for: [DanoneGTManagement] 2025-08-21 2025-09-20
Running for: [DanoneMTManagement] 2025-08-16 2025-09-15
Running for: [DelMonteManagement] 2025-09-01 2025-09-30
Running for: [EvereadyManagement] 2025-08-16 2025-09-15
Running for: [GCPLManagement] 2025-09-01 2025-09-30
Running for: [GCPL_SAMTManagement] 2025-08-21 2025-09-20
Running for: [GCPLFoodsManagement] 2025-09-01 2025-09-30
Running for: [HaleonManagement] 2025-08-26 2

In [152]:
# final_df2=final_df.drop_duplicates(['UserName','Date'])
final_df.head()

SchemaName  UserName    EmployeeID                  FullName Gender  \
0  Amul GT & MT  AMLMR015  YM2023000616         Vijayalakshmi K R      F   
1  Amul GT & MT  AMLMR027  YM2023000754  Tejaswini Sachin Shedage      F   
2  Amul GT & MT  AMLMR028  YM2023000526                 Aruna H R      F   
3  Amul GT & MT  AMLMR029  YM2023000856     Sachin Subhash Shukla      M   
4  Amul GT & MT  AMLMR040  YM2023002962                     Hamsa      M   

  Region        State       City   Designation LastWorkingDate  ...  22 23 24  \
0  South    Karnataka  Bangalore  Merchandiser      2025-08-31  ...   A  A  A   
1   West  Maharashtra     Mumbai  Merchandiser            None  ...   P  P  P   
2  South    Karnataka  Bangalore  Merchandiser      2025-10-24  ...   P  P  P   
3   West  Maharashtra     Mumbai  Merchandiser            None  ...   P  P  P   
4  South    Karnataka  Bangalore  Merchandiser            None  ...  WO  P  P   

  25 26  27  28  29 30 31  
0  A  A   A   A   A  A  A  
1  P  P   P  CO   P  P  P  
2  P  P   P   P  WO  P  P  
3  P  P  WO   P   P  P  P  
4  P  P   P   P   P  P  P  

[5 rows x 42 columns]

In [153]:
final_df2 = pd.merge(final_df,Schema_Master[['Project','From Date','To Date']], left_on = 'SchemaName' , right_on='Project', how = 'left')

In [154]:
final_df2.info() #256725
# 232885 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9502 entries, 0 to 9501
Data columns (total 45 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SchemaName       9502 non-null   object
 1   UserName         9502 non-null   object
 2   EmployeeID       9230 non-null   object
 3   FullName         9502 non-null   object
 4   Gender           9502 non-null   object
 5   Region           9502 non-null   object
 6   State            9502 non-null   object
 7   City             9502 non-null   object
 8   Designation      9448 non-null   object
 9   LastWorkingDate  1536 non-null   object
 10  EmpStatus        9502 non-null   object
 11  1                8650 non-null   object
 12  2                8673 non-null   object
 13  3                8711 non-null   object
 14  4                8727 non-null   object
 15  5                8736 non-null   object
 16  6                8799 non-null   object
 17  7                8716 non-null   

In [155]:
final_df_uservisit = pd.DataFrame()

In [156]:
for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
    uservisitquery = f"""
WITH CTE AS (
    SELECT
        '{project}' AS SchemaName,
        (TXT.UserName + '-' + TXT.Visit_Date) AS UserVisitKey,
        TXT.Visit_Date,
        TXT.UserName,
        (TXT.Visit_Date + ' ' + TXT.CHECKIN_TIME)  AS CheckinTime,
        TXT.LISStoreCode,
        (TXT.Visit_Date + '-' + TXT.LISStoreCode + '-' + TXT.UserName) AS StoreDateKey,
        TXT.StoreName,
        TXT.Designation,
        (TXT.Visit_Date + ' ' + TXT.CHECKOUT_TIME) AS CheckoutTime,
        TXT.FullName,
        TXT.NoPermissionReason,
        1 AS VisitFlag,
        DATEDIFF(
            DAY, 
            CAST('{date_from}' AS DATE), 
            TRY_CONVERT(date, TXT.Visit_Date, 105)
        ) + 1 AS DayNum
        -- DATEPART(DAY, TRY_CONVERT(date, TXT.Visit_Date, 105)) AS DayNum
    FROM (
        SELECT
            FORMAT(CONVERT(datetime, SA.[VisitDate], 105), 'dd-MM-yyyy') AS Visit_Date,
            SA.LISStoreCode,
            SA.StoreName,
            SA.UserName,
            SA.Designation,
            SA.FullName,
            CONVERT(varchar(8), CAST([CheckIn-Time]  AS time), 108) AS CHECKIN_TIME,
            CONVERT(varchar(8), CAST([CheckOut-Time] AS time), 108) AS CHECKOUT_TIME,
            SA.NoPermissionReason
        FROM [PowerBI_LISV2].{value}.[ISPStoreAttendance_NormalExport] AS SA
        LEFT JOIN (
            SELECT K.*
            FROM (
                SELECT *,
                       RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                FROM PowerBI_LISV2.{value}.UserMaster
            ) AS K
            WHERE K.RNK = 1
        ) AS UM
            ON UM.UserName = SA.UserName
        WHERE 
            TRY_CONVERT(date, SA.[VisitDate], 105) BETWEEN '{date_from}' AND '{date_to}'
            -- [Month] = MONTH(EOMONTH(GETDATE()))
            -- AND [Year]  = YEAR(EOMONTH(GETDATE()))
			-- True is considered here for all active users 
            -- AND UM.Status = 'True'
            AND SA.UserName NOT LIKE '%Star%'
            AND LOWER(SA.UserName) NOT LIKE '%test%'
    ) AS TXT
),
-- FILTERED is a table created to group by the user visit count and created a unique visit flag
Filtered AS (
    SELECT  
        -- CTE.UserVisitKey,
		CTE.UserName,
        CTE.DayNum,
        CASE WHEN COUNT(*) >= 1 THEN 1 ELSE 0 END AS VisitFlag
    FROM CTE
    GROUP BY 
	-- CTE.UserVisitKey,
	CTE.UserName, CTE.DayNum
),
-- creaing a pivot for day numbers at unique user level
Pivoted AS (
    SELECT
        UserName,
        [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
        [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
        [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
    FROM Filtered
    PIVOT (
        MAX(VisitFlag) FOR DayNum IN (
            [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
            [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
            [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
        )
    ) AS P
)
-- selecting from pivoted table and creating mtd total columns
SELECT 
    UserName,
    [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
    [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
    [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31],
    -- Month-to-date total visits for the filtered month
    ISNULL([1],0)+ISNULL([2],0)+ISNULL([3],0)+ISNULL([4],0)+ISNULL([5],0)+
    ISNULL([6],0)+ISNULL([7],0)+ISNULL([8],0)+ISNULL([9],0)+ISNULL([10],0)+
    ISNULL([11],0)+ISNULL([12],0)+ISNULL([13],0)+ISNULL([14],0)+ISNULL([15],0)+
    ISNULL([16],0)+ISNULL([17],0)+ISNULL([18],0)+ISNULL([19],0)+ISNULL([20],0)+
    ISNULL([21],0)+ISNULL([22],0)+ISNULL([23],0)+ISNULL([24],0)+ISNULL([25],0)+
    ISNULL([26],0)+ISNULL([27],0)+ISNULL([28],0)+ISNULL([29],0)+ISNULL([30],0)+
    ISNULL([31],0) AS MTD_Total
FROM Pivoted
ORDER BY UserName;"""
    df = pd.read_sql(uservisitquery, conn)
    final_df_uservisit = pd.concat([final_df_uservisit, df], ignore_index=True)
    print(f"Running for: {value} {date_from} {date_to}")


Running for: [AmulManagement] 2025-08-21 2025-09-20
Running for: [AdaniWilmerManagement] 2025-08-16 2025-09-15


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_28160\2593262159.py:101: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df_uservisit = pd.concat([final_df_uservisit, df], ignore_index=True)


Running for: [BacardilegacyManagement] 2025-09-01 2025-09-30
Running for: [BajajMTManagement] 2025-08-26 2025-09-25
Running for: [BajajConsumerManagement] 2025-08-26 2025-09-25
Running for: [IDAMManagement] 2025-08-26 2025-09-25
Running for: [BrillonMTManagement] 2025-08-16 2025-09-15
Running for: [CamlinManagement] 2025-08-26 2025-09-25
Running for: [CholayilManagement] 2025-08-21 2025-09-20
Running for: [DaburPharmaManagement] 2025-09-01 2025-09-30
Running for: [DanoneGTManagement] 2025-08-21 2025-09-20
Running for: [DanoneMTManagement] 2025-08-16 2025-09-15
Running for: [DelMonteManagement] 2025-09-01 2025-09-30
Running for: [EvereadyManagement] 2025-08-16 2025-09-15
Running for: [GCPLManagement] 2025-09-01 2025-09-30
Running for: [GCPL_SAMTManagement] 2025-08-21 2025-09-20
Running for: [GCPLFoodsManagement] 2025-09-01 2025-09-30
Running for: [HaleonManagement] 2025-08-26 2025-09-25
Running for: [HimalayaTurmericGTManagement] 2025-08-21 2025-09-20
Running for: [HimalayaTurmericManag

In [157]:
# final_df.to_csv(r'C:\Users\Administrator\LIS\MISAutomation\Kunal\payrollattendance.csv')

In [158]:
final_df_uservisit=final_df_uservisit.fillna(0)

In [159]:
final_df_uservisit.head()

UserName    1    2    3    4    5    6    7    8    9  ...   23   24   25  \
0  AMLMR015  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1  AMLMR027  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  ...  1.0  1.0  1.0   
2  AMLMR028  1.0  1.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0  ...  0.0  1.0  1.0   
3  AMLMR029  1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0  ...  1.0  1.0  1.0   
4  AMLMR040  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  1.0  1.0   

    26   27   28   29   30   31  MTD_Total  
0  0.0  0.0  0.0  0.0  0.0  0.0          2  
1  1.0  1.0  0.0  1.0  1.0  1.0         26  
2  1.0  1.0  1.0  0.0  1.0  1.0         24  
3  1.0  0.0  1.0  1.0  1.0  1.0         26  
4  1.0  1.0  1.0  0.0  1.0  1.0         22  

[5 rows x 33 columns]

In [160]:
final_df3=pd.merge(final_df2,final_df_uservisit[['UserName','MTD_Total']], on = 'UserName', how='left')

In [161]:
# final_df.to_csv(r'C:\Users\Administrator\LIS\MISAutomation\Kunal\payrollattendancepivot_df.csv')

In [162]:
#Sending df on sheet
Data.batch_clear(["A1"])
set_with_dataframe(Data, final_df3)